In [13]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes
sns.set()

In [14]:
# import the necessary keras packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.dogs_vs_cats.config import dog_vs_cats_config as config
from loader_util.preprocessing import ImageToArrayPreprocessor, \
    AspectAwarePreprocessor, SimplePreProcessor, PatchPreprocessor, \
    CropPreprocessor, MeanPreprocessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.io import HDF5DatasetGenerator
from loader_util.nn.conv import FCHeadNet, AlexNet
from loader_util.callbacks import TrainingMonitor
##
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD, Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from imutils import paths
import json
import os

In [15]:
# construct the image generator
aug = ImageDataGenerator(rotation_range=20,
                         zoom_range=0.15,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         shear_range=0.15,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [16]:
# load the RGB means for the trainig set
means = json.loads(open(config.DATASET_MEAN).read())

# initlaise the preprocessors
sp = SimplePreProcessor(227, 227)
mp = MeanPreprocessor(rmean=means["R"], gmean=means["G"], bmean=means["B"])
pp = PatchPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

In [17]:
# init the data generators
traingen = HDF5DatasetGenerator(dbPath=config.TRAIN_HDF5,
                                batchSize=128,
                                preprocessors=[pp, mp, iap],
                                classes=2)
validgen = HDF5DatasetGenerator(dbPath=config.VALID_HDF5,
                                batchSize=128,
                                preprocessors=[sp, mp, iap],
                                classes=2)

In [18]:
# init the optimizer
opt = Adam(lr=0.001)
model = AlexNet.build(width=227, height=227, depth=3, classes=2)
model.compile(loss="binary_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

NameError: name 'Adam' is not defined

In [ ]:
epochs = 75
batch_size = 128
# train the network
H = model.fit_generator(traingen.generator(),
                        steps_per_epoch=traingen.numImages//batch_size,
                        validation_data=validgen.generator(),
                        validation_steps=validgen.numImages//batch_size,
                        epochs=epochs,
                        max_queue_size=20,
                        verbose=1
                        )

In [19]:
# save the model
model.save(config.MODEL_PATH, overwrite=True)
traingen.close()
validgen.close()

NameError: name 'model' is not defined